# Afriat

In [1]:
import warnings
import os
import sys
import gc
import warnings

In [2]:
import anndata as ad
import scanpy as sc
import copy
import torch
from pathlib import Path
import networkx as nx
from sklearn.neighbors import kneighbors_graph
import numpy as np
import scanpy as sc
import pandas as pd
import numpy as np
import scipy.stats
import sklearn

sys.path.append("/Users/kemalinecik/git_nosync/tardis")
import tardis
tardis.config = tardis.config_local
print(f"CUDA used: {torch.cuda.is_available()}")

CUDA used: False


In [3]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib.pyplot as plt
import seaborn as sns

import pickle
_rcparams_path = "/Users/kemalinecik/git_nosync/tardis/training/local/figures/rcparams.pickle"
with open(_rcparams_path, 'rb') as file:
    _rcparams = pickle.load(file)
plt.rcParams.update(_rcparams)

In [4]:
adata_file_path = os.path.join(tardis.config.io_directories["processed"], "biolord_afriat.h5ad")
assert os.path.isfile(adata_file_path), f"File not already exist: `{adata_file_path}`"
adata = ad.read_h5ad(adata_file_path)

In [5]:
dir_path = os.path.join(
    tardis.config.io_directories["models"],
    "afriat_v2_4"
)
vae = tardis.MyModel.load(dir_path=dir_path, adata=adata)
adata.obs["validation"] = "train"
adata.obs["validation"].iloc[vae.validation_indices] = "validation"
adata

INFO     File /Users/kemalinecik/git_nosync/tardis/data/models/afriat_v2_4/model.pt already downloaded             


/Users/kemalinecik/tools/apps/mamba/envs/tardis_env/lib/python3.10/site-packages/scvi/data/fields/_base_field.py:64: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)
/var/folders/y7/7c17s0l57szdjc1cdc9dmnpm0000gn/T/ipykernel_3226/1723890290.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adata.obs["validation"].iloc[vae.validation_indices] = "validation"


AnnData object with n_obs × n_vars = 19053 × 5000
    obs: 'mouse', 'experiment', 'time_int', 'time_cat', 'zone', 'status_control', '_scvi_batch', '_scvi_labels', 'validation'
    uns: '_scvi_uuid', '_scvi_manager_uuid'
    obsm: '_scvi_disentanglement_target', '_scvi_metrics_helper'

In [6]:
from tardis._disentanglementmanager import DisentanglementManager as DM

In [7]:
vae.get_reconstruction_r2_training(top_n=[])

{('validation', 'total'): 0.9994887866777654,
 ('train', 'total'): 0.9995524077088713}

In [8]:
vae.get_MI_normalized_training("zone")

{('validation', 'reserved'): 0.9959095223197801,
 ('validation', 'unreserved'): 0.09693034298263432,
 ('train', 'reserved'): 0.9955071531928366,
 ('train', 'unreserved'): 0.05177080394839084}

In [9]:
vae.get_MI_normalized_training("status_control")

{('validation', 'reserved'): 0.9748486760298751,
 ('validation', 'unreserved'): 0.18363561083973562,
 ('train', 'reserved'): 0.9652775732050243,
 ('train', 'unreserved'): 0.12443429868443108}

In [10]:
vae.get_MI_normalized_training("time_cat")

{('validation', 'reserved'): 0.9089229154806326,
 ('validation', 'unreserved'): 0.19244730200908963,
 ('train', 'reserved'): 0.9024865058240139,
 ('train', 'unreserved'): 0.11176398635138754}

In [11]:
vae.get_knn_purity_training("zone")

{('validation', 'reserved'): 1.0,
 ('validation', 'unreserved'): 0.6716171231648069,
 ('train', 'reserved'): 1.0,
 ('train', 'unreserved'): 0.7177828986188928}

In [12]:
vae.get_knn_purity_training("status_control")

{('validation', 'reserved'): 1.0,
 ('validation', 'unreserved'): 0.737043561219943,
 ('train', 'reserved'): 1.0,
 ('train', 'unreserved'): 0.7699448487172272}

In [13]:
vae.get_knn_purity_training("time_cat")

{('validation', 'reserved'): 1.0,
 ('validation', 'unreserved'): 0.6460812717294607,
 ('train', 'reserved'): 1.0,
 ('train', 'unreserved'): 0.7113515204479276}

In [14]:
data = vae.get_latent_representation()

factors_indices = sorted([[indice, config.obs_key] for config in DM.configurations.items for indice in config.reserved_latent_indices])
factors = np.vstack([sklearn.preprocessing.LabelEncoder().fit_transform(
    adata.obs[j].copy().astype(str).values.flatten()
) for i, j in factors_indices]).T

data = data[:, [i for i, j in factors_indices]]

In [15]:
data.shape, factors.shape

((19053, 24), (19053, 24))

In [16]:
maxMIG = vae.get_MI_precalculated(data=data, factors=factors, variation="maxMIG", discretization_bins=256)
maxMIG_mean = np.mean(maxMIG)
maxMIG_mean

0.9085402872869061

In [17]:
maxMIG = vae.get_MI_precalculated(data=data[vae.validation_indices, :], factors=factors[vae.validation_indices, :], variation="maxMIG", discretization_bins=256)
maxMIG_mean = np.mean(maxMIG)
maxMIG_mean

0.8583966652389495

In [18]:
maxMIG = vae.get_MI_precalculated(data=data[vae.train_indices, :], factors=factors[vae.train_indices, :], variation="maxMIG", discretization_bins=256)
maxMIG_mean = np.mean(maxMIG)
maxMIG_mean

0.9053053572434181

# separate

In [55]:
vaes = dict()
dir_path = os.path.join(
    tardis.config.io_directories["models"],
    "afriat_v1_1"
)
vaes["status_control"] = tardis.MyModel.load(dir_path=dir_path, adata=adata)

INFO     File /Users/kemalinecik/git_nosync/tardis/data/models/afriat_v1_1/model.pt already downloaded             


/Users/kemalinecik/tools/apps/mamba/envs/tardis_env/lib/python3.10/site-packages/scvi/data/fields/_base_field.py:64: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)
/Users/kemalinecik/tools/apps/mamba/envs/tardis_env/lib/python3.10/site-packages/scvi/model/base/_base_model.py:699: UserWarning: `(None, ['zone', 'time_cat'])` is defined as `batch_key` or `categorical_covariate`, it will be given to both encoder and decoder. Make sure this does not contain information of any of your disentenglement targets. For example, if `donor_id` is chosen as a batch key, do not disentengle donor level information such as `sex` or `age`. The decoder should not use the disentengled latent spaces, simply ignores, if it is already given a batch_key. 
  getattr(cls, method_name)(adata, source_registry=registry, **registry[_SETUP_ARGS_KEY])


In [56]:
dir_path = os.path.join(
    tardis.config.io_directories["models"],
    "afriat_v3_1"
)
vaes["zone"] = tardis.MyModel.load(dir_path=dir_path, adata=adata)

INFO     File /Users/kemalinecik/git_nosync/tardis/data/models/afriat_v3_1/model.pt already downloaded             


/Users/kemalinecik/tools/apps/mamba/envs/tardis_env/lib/python3.10/site-packages/scvi/data/fields/_base_field.py:64: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)
/Users/kemalinecik/tools/apps/mamba/envs/tardis_env/lib/python3.10/site-packages/scvi/model/base/_base_model.py:699: UserWarning: `(None, ['status_control', 'time_cat'])` is defined as `batch_key` or `categorical_covariate`, it will be given to both encoder and decoder. Make sure this does not contain information of any of your disentenglement targets. For example, if `donor_id` is chosen as a batch key, do not disentengle donor level information such as `sex` or `age`. The decoder should not use the disentengled latent spaces, simply ignores, if it is already given a batch_key. 
  getattr(cls, method_name)(adata, source_registry=registry, **registry[_SETUP_ARGS_KEY])


In [57]:
dir_path = os.path.join(
    tardis.config.io_directories["models"],
    "afriat_v4_1"
)
vaes["time_cat"] = tardis.MyModel.load(dir_path=dir_path, adata=adata)

INFO     File /Users/kemalinecik/git_nosync/tardis/data/models/afriat_v4_1/model.pt already downloaded             


/Users/kemalinecik/tools/apps/mamba/envs/tardis_env/lib/python3.10/site-packages/scvi/data/fields/_base_field.py:64: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)
/Users/kemalinecik/tools/apps/mamba/envs/tardis_env/lib/python3.10/site-packages/scvi/model/base/_base_model.py:699: UserWarning: `(None, ['status_control', 'zone'])` is defined as `batch_key` or `categorical_covariate`, it will be given to both encoder and decoder. Make sure this does not contain information of any of your disentenglement targets. For example, if `donor_id` is chosen as a batch key, do not disentengle donor level information such as `sex` or `age`. The decoder should not use the disentengled latent spaces, simply ignores, if it is already given a batch_key. 
  getattr(cls, method_name)(adata, source_registry=registry, **registry[_SETUP_ARGS_KEY])


In [58]:
for k in vaes:
    adata.obs[f"validation_{k}"] = "train"
    adata.obs[f"validation_{k}"].iloc[vaes[k].validation_indices] = "validation"

/var/folders/y7/7c17s0l57szdjc1cdc9dmnpm0000gn/T/ipykernel_3226/2097264642.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adata.obs[f"validation_{k}"].iloc[vaes[k].validation_indices] = "validation"


In [59]:
factors_indices = sorted([[k, i] for k in vaes for i in range(8)])
datas = {j: vaes[j].get_latent_representation() for j in vaes}
factors_indices_obs = sorted({i for i, j in factors_indices})


/Users/kemalinecik/tools/apps/mamba/envs/tardis_env/lib/python3.10/site-packages/scvi/data/fields/_base_field.py:64: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)


/Users/kemalinecik/tools/apps/mamba/envs/tardis_env/lib/python3.10/site-packages/scvi/data/fields/_base_field.py:64: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)
/Users/kemalinecik/tools/apps/mamba/envs/tardis_env/lib/python3.10/site-packages/scvi/data/fields/_base_field.py:64: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)


In [60]:
factors_indices_obs

['status_control', 'time_cat', 'zone']

In [61]:
factors = np.vstack([sklearn.preprocessing.LabelEncoder().fit_transform(
    adata.obs[j].copy().astype(str).values.flatten()
) for j, i in factors_indices]).T
factors.shape

(19053, 24)

In [62]:
data = np.hstack([datas[d][:, range(8)] for d in factors_indices_obs])
data.shape

(19053, 24)

In [63]:
maxMIG = vae.get_MI_precalculated(data=data, factors=factors, variation="maxMIG", discretization_bins=256)
maxMIG_mean = np.mean(maxMIG)
maxMIG_mean

0.9141959701135955

In [64]:
maxMIG = vae.get_MI_precalculated(data=data[vaes["zone"].validation_indices, :], factors=factors[vaes["zone"].validation_indices, :], variation="maxMIG", discretization_bins=256)
maxMIG_mean = np.mean(maxMIG)
maxMIG_mean

0.8681261580573579

In [65]:
maxMIG = vae.get_MI_precalculated(data=data[vaes["time_cat"].validation_indices, :], factors=factors[vaes["time_cat"].validation_indices, :], variation="maxMIG", discretization_bins=256)
maxMIG_mean = np.mean(maxMIG)
maxMIG_mean

0.8697035821646785

In [66]:
maxMIG = vae.get_MI_precalculated(data=data[vaes["status_control"].validation_indices, :], factors=factors[vaes["status_control"].validation_indices, :], variation="maxMIG", discretization_bins=256)
maxMIG_mean = np.mean(maxMIG)
maxMIG_mean

0.8663798208835223